<a href="https://colab.research.google.com/github/IkeLyons/HW4-Sequence-Models/blob/main/SequenceModels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this homework I am going to try to generate text based off of the script of Star Wars (dataset link: https://www.kaggle.com/xvivancos/star-wars-movie-scripts). This problem could not be solved without the use of a sequential model due to the nature of text generation. When you are generating text the next character that is typed depends on all of the ones before it. This can be easily accomplished by sequence models such as a RNN that factor in a timestep. 

In [1]:
! pip install kaggle

  Using cached kaggle-1.5.12.tar.gz (58 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached python_slugify-5.0.2-py2.py3-none-any.whl (6.7 kB)
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73052 sha256=86e6578ed4d87a869783a280601513e2cf84ef86bea8e0eae5372518c11ac540
  Stored in directory: c:\users\ikely\appdata\local\pip\cache\wheels\ac\b2\c3\fa4706d469b5879105991d1c8be9a3c2ef329ba9fe2ce5085e
Successfully built kaggle


In [2]:
! mkdir ~/.kaggle

The syntax of the command is incorrect.


In [3]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
! kaggle datasets download xvivancos/star-wars-movie-scripts

Traceback (most recent call last):
  File "c:\python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Python39\Scripts\kaggle.exe\__main__.py", line 4, in <module>
  File "c:\python39\lib\site-packages\kaggle\__init__.py", line 23, in <module>
    api.authenticate()
  File "c:\python39\lib\site-packages\kaggle\api\kaggle_api_extended.py", line 164, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in C:\Users\ikely\.kaggle. Or use the environment method.


In [5]:
! unzip -o star-wars-movie-scripts.zip 

'unzip' is not recognized as an internal or external command,
operable program or batch file.


We are using TensorFlow and some of the Keras utilties in order to solve this problem.

In [6]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np

In [8]:
text = open('SW_EpisodeIV.txt', 'rb').read().decode(encoding='utf-8')
text = text.lower()
text = text.replace('"', '')
print(text[:500])

character dialogue
1 threepio did you hear that?  they've shut down the main reactor.  we'll be destroyed for sure.  this is madness!
2 threepio we're doomed!
3 threepio there'll be no escape for the princess this time.
4 threepio what's that?
5 threepio i should have known better than to trust the logic of a half-sized thermocapsulary dehousing assister...
6 luke hurry up!  come with me!  what are you waiting for?!  get in gear!
7 threepio artoo! artoo-detoo, where are you?
8 threepio at last! 


We can see that the script has 3 main parts: the line number, the character speaking, and what the character said. It will be interesting to see how the models are able to recreate this.

In [9]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', "'", ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '?', '\\', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


# Task 1

Now we need to transform this script data and character encoding, into the actual datasets that we are going to use for training.

In [10]:
import keras as K

seq_length = 50
X_train = []
y_train = []
# X_train needs to be an array of the indicies of the characters in the sequence
# y_train needs to be the correct next character
for i in range(0, len(text) - seq_length):
  X = text[i:i + seq_length]
  y = text[i + seq_length]
  X_train.append([chars.index(x) for x in X])
  y_train.append(chars.index(y))

X_train = np.reshape(X_train, (len(X_train), seq_length))
# convert to one hot encoding
y_train = K.utils.np_utils.to_categorical(y_train)

print(X_train[0])
print(y_train[0])
print(''.join([chars[x] for x in X_train[0]]))

[22 27 20 37 20 22 39 24 37  1 23 28 20 31 34 26 40 24  0  8  1 39 27 37
 24 24 35 28 34  1 23 28 23  1 44 34 40  1 27 24 20 37  1 39 27 20 39 18
  1  1]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
character dialogue
1 threepio did you hear that?  


Doing the Training, Validation, Test split

In [11]:
from sklearn.model_selection import train_test_split 

# train: 80%, val: 10%, test: 10%
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=1) 

# Task 1 (Part 1)

Now that our dataset is ready, it is time to create the actual model. I am using a structure of one input layer connected to an embedding layer whose size is a huyperparameter. That layer outputs a 3D tensor that is fed into to the SimpleRNN layer which is what handles that extra timestep dimension. That is then connected to the dense layer which feeds into the output. In the dense layer I am using an exponential linear unit activation function. In the output layer I am using softmax. Finally, I opt to use the adam optimizer for the whole thing and use categorical cross entropy to find the the loss.

In [12]:
rnn_units = 256
dense_units = 128
embedding_dim = 100

model = tf.keras.models.Sequential()
input = tf.keras.layers.Input(shape=(seq_length, ))
embedding = tf.keras.layers.Embedding(len(chars), embedding_dim, trainable=False)(input)
simple_rnn = tf.keras.layers.SimpleRNN(rnn_units,)(embedding)
dense = tf.keras.layers.Dense(dense_units, activation="elu")(simple_rnn)
output = tf.keras.layers.Dense(len(chars), activation='softmax')(dense)
    
model = tf.keras.Model(inputs=input, outputs=output)

In [13]:
model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy'],
              )

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 50, 100)           4600      
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 256)               91392     
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 46)                5934      
Total params: 134,822
Trainable params: 130,222
Non-trainable params: 4,600
_________________________________________________________________


In [15]:
print([chars[x] for x in X_train[0]])

['o', 'n', '.', ' ', ' ', 'y', 'o', 'u', ' ', 'm', 'a', 'y', ' ', 'f', 'i', 'r', 'e', ' ', 'w', 'h', 'e', 'n', ' ', 'r', 'e', 'a', 'd', 'y', '.', '\n', '4', '3', '6', ' ', 'l', 'e', 'i', 'a', ' ', 'w', 'h', 'a', 't', '?', '\n', '4', '3', '7', ' ', 't']


Now we can setup some callback functions so that the model will be able to save itself at checkpoints and print out some sample text at the end of every epoch.

In [16]:
import os
from keras.callbacks import Callback

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

class TextGeneration(Callback):
    def __init__(self):
       super(Callback, self).__init__() 

    def on_epoch_end(self, epoch, logs={}):
      train_indicies = X_train[0]
      train_text = [chars[x] for x in X_train[0]]
      print("==========================================================")
      print("Seed Text: " + ''.join(train_text))
      # generate another 50 characters based off the seed indicies
      predicted_chars = []
      for i in range(50):
        pred = self.model.predict(np.reshape(train_indicies, (1, len(train_indicies))))
        result = np.argmax(pred)
        predicted_chars.append(chars[result])
        #we add the result to the end of the training set so that the next predicition uses it
        train_indicies = np.append(train_indicies, result)
        train_indicies = train_indicies[1:len(train_indicies)]
      print("Generated Text: " + ''.join(predicted_chars))
      print("==========================================================")

text_generation = TextGeneration()

In [17]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=2,
          callbacks=[checkpoint_callback, text_generation])

Epoch 1/40
902/902 - 54s - loss: 2.6217 - accuracy: 0.2759
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: he the the the the the the the the the the the the
Epoch 2/40
902/902 - 51s - loss: 2.2094 - accuracy: 0.3484
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: an the the the sting the pit the sting the pit the
Epoch 3/40
902/902 - 52s - loss: 2.0938 - accuracy: 0.3749
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepio thing the thing the thing the thing the th
Epoch 4/40
902/902 - 47s - loss: 2.0030 - accuracy: 0.3957
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: his ster the stan the stan the stan the stan the s
Epoch 5/40
902/902 - 48s - loss: 1.9277 - accuracy: 0.4193
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepio the the to the sore ind the sore ind the s
Epoch 6/40
902/902 - 47s - loss: 1.8602 - accuracy: 0.4358
S

In order to evaluate our model we are going to look at the accuracy precentage of the model on the training data, and we are going to subjectivley view the output that the model is producing and compare betweeen changes. This will be done by running an evaluation method on the validation set first, then the test set. Im then also going to generate 200 characters of text based off seed data from the test set so that I can have generated output to compare.

In [18]:
model.evaluate(X_val, y_val,
          batch_size=64,
          verbose=2,
          callbacks=[checkpoint_callback, text_generation])

113/113 - 3s - loss: 1.6675 - accuracy: 0.5270


[1.6674785614013672, 0.5270232558250427]

In [19]:
model.evaluate(X_test, y_test,
          batch_size=64,
          verbose=2,
          callbacks=[checkpoint_callback, text_generation])

113/113 - 3s - loss: 1.6990 - accuracy: 0.5176


[1.6989578008651733, 0.5175997614860535]

In [20]:
train_indicies = X_test[0]
train_text = [chars[x] for x in X_test[0]]
print("==========================================================")
print("Seed Text: " + ''.join(train_text))
# generate another 200 characters based off the seed indicies
predicted_chars = []
for i in range(200):
  pred = model.predict(np.reshape(train_indicies, (1, len(train_indicies))))
  result = np.argmax(pred)
  predicted_chars.append(chars[result])
  #we add the result to the end of the training set so that the next predicition uses it
  train_indicies = np.append(train_indicies, result)
  train_indicies = train_indicies[1:len(train_indicies)]
print("Generated Text: " + ''.join(predicted_chars))
print("==========================================================")

Seed Text: 
274 luke i can't get involved!  i've got work to 
Generated Text: the ship the planet in there is my procise i there?
559 ben it should have to going to flong to go about it.
703 leia their minder your sorten in the planet in there is my procise i there?
559 ben it 


# Task 1 (Part 2)

The only changes I have made to the model below is changing the SimpleRNN layer to an LSTM.

In [21]:
rnn_units = 256
dense_units = 128
embedding_dim = 100

model = tf.keras.models.Sequential()
input = tf.keras.layers.Input(shape=(seq_length, ))
embedding = tf.keras.layers.Embedding(len(chars), embedding_dim, trainable=False)(input)
LSTM = tf.keras.layers.LSTM(rnn_units,)(embedding)
dense = tf.keras.layers.Dense(dense_units, activation="elu")(LSTM)
output = tf.keras.layers.Dense(len(chars), activation='softmax')(dense)
    
model = tf.keras.Model(inputs=input, outputs=output)

In [22]:
model.compile(loss='categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                metrics=['accuracy'],
              )

In [23]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 50)]              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 100)           4600      
_________________________________________________________________
lstm (LSTM)                  (None, 256)               365568    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 46)                5934      
Total params: 408,998
Trainable params: 404,398
Non-trainable params: 4,600
_________________________________________________________________


In [ ]:
model.fit(X_train, y_train,
          batch_size=64,
          epochs=40,
          verbose=2,
          callbacks=[checkpoint_callback, text_generation])

Epoch 1/40
902/902 - 229s - loss: 2.6433 - accuracy: 0.2669
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepin the the the the the the the the the the th
Epoch 2/40
902/902 - 218s - loss: 2.0979 - accuracy: 0.3768
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepio on the pere the sed the sed the sed the se
Epoch 3/40
902/902 - 211s - loss: 1.9067 - accuracy: 0.4263
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepio i'm in't the seation the seater an the sea
Epoch 4/40
902/902 - 210s - loss: 1.7787 - accuracy: 0.4573
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepio what you see the seation the reate the mas
Epoch 5/40
902/902 - 206s - loss: 1.6707 - accuracy: 0.4875
Seed Text: on.  you may fire when ready.
436 leia what?
437 t
Generated Text: hreepio i could i way here the ship ship ship she 
Epoch 6/40
902/902 - 206s - loss: 1.5740 - accuracy: 0.

In [ ]:
model.evaluate(X_val, y_val,
          batch_size=64,
          verbose=2,
          callbacks=[checkpoint_callback, text_generation])

In [ ]:
model.evaluate(X_test, y_test,
          batch_size=64,
          verbose=2,
          callbacks=[checkpoint_callback, text_generation])

In [ ]:
train_indicies = X_test[0]
train_text = [chars[x] for x in X_test[0]]
print("==========================================================")
print("Seed Text: " + ''.join(train_text))
# generate another 200 characters based off the seed indicies
predicted_chars = []
for i in range(200):
  pred = model.predict(np.reshape(train_indicies, (1, len(train_indicies))))
  result = np.argmax(pred)
  predicted_chars.append(chars[result])
  #we add the result to the end of the training set so that the next predicition uses it
  train_indicies = np.append(train_indicies, result)
  train_indicies = train_indicies[1:len(train_indicies)]
print("Generated Text: " + ''.join(predicted_chars))
print("==========================================================")

The major differences are that the LSTM model takes much longer to train and ended with far higher accuracy after training when compared to the SimpleRNN model. Despite the major difference in final training accuracy ( 95% vs. 62% ) the accuracy on the validation and test sets was very similar to one another, and thus it seems the two may generalize the same. When comparing the output of the two, the lstm seems to use puncutuation better and less of the words are made up. Its difficult to say if this is due to randomness or because the LSTM model learned these things better, but with trying different seeds this seems to hold generally true.

# Task 2

Below I load the Google's pre-trained word2vec (https://code.google.com/archive/p/word2vec/)

In [ ]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
p1w1 = w2v[input("Enter pair 1 word 1: ")]
p1w2 = w2v[input("Enter pair 1 word 2: ")]
p2w1 = w2v[input("Enter pair 2 word 1: ")]
p2w2 = w2v[input("Enter pair 2 word 2: ")]

Enter pair 1 word 1: dog
Enter pair 1 word 2: puppy
Enter pair 2 word 1: bottle
Enter pair 2 word 2: hat


In [ ]:
from numpy import dot
from numpy.linalg import norm
import math
# cosine similarity implemented using numpy
def cosine_similarity(list1, list2):
  result = dot(list1, list2)/(norm(list1)*norm(list2))
  return(result)

def euclidian_dissimilarity(list1, list2):
  sum = 0
  for i in range(len(list1)):
    sum += (list1[i] - list2[i])**2
  return math.sqrt(sum)

sim1 = str(cosine_similarity(p1w1, p1w2))
sim2 = str(cosine_similarity(p2w1, p2w2))
sim3 = str(euclidian_dissimilarity(p1w1, p1w2))
sim4 = str(euclidian_dissimilarity(p2w1, p2w2))
print("cosine similarity of the first pair of words is: " + sim1)
print("cosine similarity of the second pair of words is: " + sim2)
print("euclidian dissimilarity of the first pair of words is: " + sim3)
print("euclidian dissimilarity of the second pair of words is: " + sim4)

cosine similarity of the first pair of words is: 0.8106429
cosine similarity of the second pair of words is: 0.2739666
euclidian dissimilarity of the first pair of words is: 1.9458779775316037
euclidian dissimilarity of the second pair of words is: 3.7057712646698056


The measure of dissimiliarity that I implemented is the euclidian distance between the word embedings as described in [An Empirical Comparison of Dissimilarity Measures for 
Recommender Systems](https://poseidon01.ssrn.com/delivery.php?ID=208022116002080089004123066029071081070087090027053041122038061059001002112098072115067106070047030079115127070066105085067124127006065004087015115079088121002111114022116005022&EXT=pdf&INDEX=TRUE). The reason that euclidian distance makes sense as a disimilarity score is that it is essentially just measures the distance between all the word embedding values. If the word embedding values arent very different (i.e. the difference between them is closer to 0) then the euclidian distance will be lower and thus the dissimilarity score. 